In [12]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:.4f}".format)

input_data_path = "../data/input_data.xlsx"
final_data_path = "../data/final_data.xlsx"
imputed_data_path = "../data/imputed_data.xlsx"
nutrition_data_path = "../data/nutrition_data.xlsx"

input_df = pd.ExcelFile(input_data_path).parse(sheet_name="Sheet1")
final_df = pd.ExcelFile(final_data_path).parse(sheet_name="Sheet2")
imputed_df = pd.ExcelFile(imputed_data_path).parse(sheet_name="Sheet1") # (341, 38)
nutrition_df = pd.ExcelFile(nutrition_data_path).parse(sheet_name="Sheet1") # (2191, 64)

In [13]:
# GẮN CỘT NUMBER VÀO IMPUTED DATA

imputed_df.insert(0, "sub_id", final_df["number"])

In [14]:
# XEM QUA CÁC CẤU PHẦN TPLS VÀ CHỨA CHỮ

tpls_ids = nutrition_df.loc[nutrition_df["Number"].astype(str).str.contains("TPLS", case=False, na=False), "Number"].tolist()
c_ids = nutrition_df.loc[nutrition_df["Number"].astype(str).str.match(r"^c", case=False, na=False), "Number"].tolist()


In [15]:
# LỌC BỎ PHẦN TPLS_

nutrition_df.loc[nutrition_df["Number"].str.contains(r"^TPLS_", case=False, na=False), 
    "Number"
] = nutrition_df["Number"].str.replace(r"^TPLS_", "", regex=True)

In [16]:
# XEM CÁC ID CÓ ÍT HƠN 4 KÍ TỰ Ở NUTRITION

nutrition_side = nutrition_df.loc[nutrition_df["Number"].str.len() < 4, "Number"].to_list()

# nutrition_side

In [17]:
# XEM CÁC ID CÓ ÍT HƠN 4 KÍ TỰ Ở IMPUTED

imputed_side = imputed_df.loc[imputed_df["sub_id"].str.len() < 4, "sub_id"].to_list()

# imputed_side

In [18]:
# XEM CÁC NHÓM GIAO 

ids_imputed = imputed_df["sub_id"].unique()
ids_nutrition = nutrition_df["Number"].unique()

ids_imputed_set = set(ids_imputed)
ids_nutrition_set = set(ids_nutrition)
common_ids = ids_imputed_set.intersection(ids_nutrition_set)

# len(common_ids) # có 237 giá trị chung
# common_ids

In [19]:
# VẤN ĐỀ SỐ 0 TRONG NUTRITION DF 

zero_begin_nu = nutrition_df.loc[nutrition_df["Number"].astype(str).str.match(r"^0", case=False, na=False), "Number"].tolist()
target_ls = ["417", "959", "998", "518"]
# target_im = imputed_df.loc[imputed_df["number"].str.contains(r"(0417|0959|0998|0518)", case=False, na=False), "number"].to_list()
target_im = imputed_df.loc[imputed_df["sub_id"].str.contains(r"(417|959|998|518)", case=False, na=False), "sub_id"].to_list()
# target_im # dù theo cách nào thì cũng trả về một list rỗng 

C:\Users\namtu\AppData\Local\Temp\ipykernel_4568\3852121351.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  target_im = imputed_df.loc[imputed_df["sub_id"].str.contains(r"(417|959|998|518)", case=False, na=False), "sub_id"].to_list()


In [20]:
# VẤN ĐỀ SỐ 0 TRONG IMPUTED DF 

zero_begin_im = imputed_df.loc[imputed_df["sub_id"].astype(str).str.match(r"^0", case=False, na=False), "sub_id"].tolist()
zero_begin_im # không có bất kì number nào trong imputed_df có chứa số 0 ở đầu tiên 
# như vậy, số 0 đầu tiên không phải là vấn đề


[]

In [21]:
imputed_df.columns

Index(['sub_id', 'age', 'zbmi', 'insulin', 'glucose_ac', 'cholesterol', 'TG',
       'HDL', 'cortisol', 'waist', 'hip', 'SBP', 'DBP', 'CPP', 'sex',
       'family_income', 'pregnancy_smoking', 'GDM', 'gestational_weight_gain',
       'preterm_birth', 'gestational_age_week', 'birth_weight_gram',
       'exclusive_breastfeeding_month', 'mixed_breastfeeding_month',
       'father_diabetes', 'mother_diabetes', 'education_level', 'father_BMI',
       'mother_BMI', 'sedentary_lifestyle_hour_day',
       'low_physical_activity_hour_day', 'sleep_duration', 'PSQI_score',
       'snoring_times_week', 'sleep_apnea_times_week', 'score_5A', 'PSS',
       'score_5B', 'CES_D', 'score_5C', 'RSE'],
      dtype='object')

In [24]:
# CHUẨN BỊ IMPUTED_SUBSET

removed_cols = ["TG", "HDL", "cortisol", "waist", "hip", "SBP", "DBP"]

imputed_subset = imputed_df.drop(columns=removed_cols).copy()

imputed_subset["insulin_re"] = imputed_subset.apply(
    lambda x: "no" if (x["insulin"] * x["glucose_ac"]) / 405 <= 3.16 else "yes",
    axis=1
)

imputed_subset.drop(columns=["insulin", "glucose_ac"], inplace=True)

imputed_subset.columns

Index(['sub_id', 'age', 'zbmi', 'cholesterol', 'CPP', 'sex', 'family_income',
       'pregnancy_smoking', 'GDM', 'gestational_weight_gain', 'preterm_birth',
       'gestational_age_week', 'birth_weight_gram',
       'exclusive_breastfeeding_month', 'mixed_breastfeeding_month',
       'father_diabetes', 'mother_diabetes', 'education_level', 'father_BMI',
       'mother_BMI', 'sedentary_lifestyle_hour_day',
       'low_physical_activity_hour_day', 'sleep_duration', 'PSQI_score',
       'snoring_times_week', 'sleep_apnea_times_week', 'score_5A', 'PSS',
       'score_5B', 'CES_D', 'score_5C', 'RSE', 'insulin_re'],
      dtype='object')

In [ ]:
# ĐÁNH GIÁ MỨC ĐỘ THIẾU DỮ LIỆU DINH DƯỠNG 

nutrition_cols = nutrition_df.columns

high_missing_cols = []

for col in nutrition_cols: 
    missing_percent = (nutrition_df[col].isna().mean() * 100).round(2)
    if missing_percent > 40:
        high_missing_cols.append(col)
    # print(f"The {col} column has a missing data rate of {missing_percent}%.")


In [ ]:
# ĐIỀN GIÁ TRỊ THIẾU TRONG DỮ LIỆU DINH DƯỠNG BẰNG MEDIAN 

nutrition_df = nutrition_df.fillna(nutrition_df.median(numeric_only=True))

# chú ý cách lập luận tại sao lại chọn điền bằng median

In [ ]:
# ĐỔI TÊN BẢNG DỮ LIỆU DINH DƯỠNG 

column_names_map = {
    "Number": "sub_id",
    "Energy(kcal)": "energy_kcal", 
    "Crude_Protein(g)": "crude_protein_g", 
    "Crude_Fat(g)": "crude_fat_g", 
    "Total_Carbohydrate(g)": "total_carbohydrate_g", 
    "Water(g)": "water_g", 
    "Fruits": "fruits", 
    "Vegetables": "vegetables", 
    "WholeGrainsAndRoots": "whole_grains_roots", 
    "protein(LowFat)": "protein_low_fat", 
    "protein(MediumFat)": "protein_medium_fat", 
    "protein(HighFat)": "protein_high_fat", 
    "protein(SuperHighFat)": "protein_superhigh_fat", 
    "Dairy(Skim)": "dairy_skim", 
    "Dairy(LowFat)": "dairy_low_fat", 
    "Dairy(WholeFat)": "dairy_whole_fat", 
    "Oils_Nuts_And_Seeds": "oils_nuts_and_seeds", 
    "Total_Polyunsaturated_Fatty_Acids(g)": "total_polyunsaturated_fatty_acids_g", 
    "Total_Monounsaturated_Fatty_Acids(g)": "total_monounsaturated_fatty_acids_g", 
    "Total_Saturated_Fatty_Acids(g)": "total_saturated_fatty_acids_g", 
    "CrudeFiber(g)": "crude_fiber_g", 
    "Dietary_Fiber(g)": "dietary_fiber_g", 
    "Total_Sugar(g)": "total_sugar_g", 
    "Glucose(g)": "glucose_g", 
    "Fructose(g)": "fructose_g", 
    "Maltose(g)": "maltose_g", 
    "Sucrose(g)": "sucrose_g",
    "Lactose(g)": "lactose_g", 
    "Cholesterol(mg)": "cholesterol_mg", 
    "Sodium(mg)": "sodium_mg", 
    "Potassium(mg)": "potassium_mg", 
    "Calcium(mg)": "calcium_mg", 
    "Magnesium(mg)": "magnesium_mg", 
    "Phosphorus(mg)": "phosphorus_mg", 
    "Iron(mg)": "iron_mg", 
    "Zinc(mg)": "zinc_mg", 
    "VitaminB1(Thiamin)(mg)": "vitamin_b1_mg", 
    "VitaminB2(Riboflavin)(mg)": "vitamin_b2_mg", 
    "Niacin(mg)": "niacin_mg", 
    "VitaminB6(mg)": "vitamin_b6_mg", 
    "VitaminB12(ug)": "vitamin_b12_ug", 
    "FolicAcid(ug)": "folicacid_ug", 
    "VitaminC(mg)": "vitaminc_mg",
    "TotalVitaminA(I.U.)": "totalVitaminA_ui", 
    "TotalVitaminE(mg)": "totalVitaminE_mg", 
    "trans_fat": "trans_fat", 
    "Threonine(mg)": "threonine_mg", 
    "Valine(mg)": "valine_mg", 
    "Methionine(mg)": "methionine_mg", 
    "Isoleucine(mg)": "isoleucine_mg", 
    "Leucine(mg)": "leucine_mg", 
    "Phenylalanine(mg)": "phenylalanine_mg", 
    "Lysine(mg)": "lysine_mg", 
    "Histidine(mg)": "histidine_mg", 
    "Tryptophan(mg)": "tryptophan_mg", 
    "Alanine(mg)": "alanine_mg", 
    "Arginine(mg)": "arginine_mg", 
    "AsparticAcid(mg)": "aspartic_mg", 
    "GlutamicAcid(mg)": "glutamic_mg", 
    "Glycine(mg)": "glycine_mg", 
    "Proline(mg)": "proline_mg", 
    "Serine(mg)": "serine_mg", 
    "Tyrosine(mg)": "tyrosine_mg", 
    "Cystine(mg)": "cystine_mg"           
}

nutrition_df.rename(columns=column_names_map, inplace=True)

# len(column_names_map) # 63 không tính cột number

In [ ]:
# MỘT SỐ CỘT CÓ VẤN ĐỀ

nutrition_df.drop(columns=["total_sugar_g", "crude_fiber_g"], inplace=True) # hai cột trên không có trong biểu đồ tròn
# biến Cu (Copper) có trong biểu đồ tròn nhưng không có trong cơ sở dữ liệu 
# biến calories trong biểu đồ tròn lấy giá trị của cột nào: cột energy_kcal trong nutrition_df hay cột calories(kcal) trong imputed_subset


In [ ]:
# GỘP MỘT SỐ CỘT ĐỂ THÀNH FAMILY 

# EAA 
nutrition_df["EAA"] = nutrition_df["threonine_mg"] + nutrition_df["valine_mg"] + nutrition_df["methionine_mg"] + nutrition_df["isoleucine_mg"] + nutrition_df["leucine_mg"] + nutrition_df["phenylalanine_mg"] + nutrition_df["lysine_mg"] + nutrition_df["histidine_mg"] + nutrition_df["tryptophan_mg"] 

# NEAA
nutrition_df["NEAA"] = nutrition_df["alanine_mg"] + nutrition_df["arginine_mg"] + nutrition_df["aspartic_mg"] + nutrition_df["glutamic_mg"] + nutrition_df["glycine_mg"] + nutrition_df["proline_mg"] + nutrition_df["serine_mg"] + nutrition_df["tyrosine_mg"] + nutrition_df["cystine_mg"]

# PLMF
nutrition_df["PLMF"] = nutrition_df["protein_low_fat"] + nutrition_df["protein_medium_fat"]

# PHSF 
nutrition_df["PHSF"] = nutrition_df["protein_high_fat"] + nutrition_df["protein_superhigh_fat"]

# monosaccharides
nutrition_df["monosaccharides"] = nutrition_df["glucose_g"] + nutrition_df["fructose_g"]

# disaccharides
nutrition_df["disaccharides"] = nutrition_df["maltose_g"] + nutrition_df["sucrose_g"] + nutrition_df["lactose_g"]

# dairy (skim & lowfat)
nutrition_df["dairy_skim_low_fat"] = nutrition_df["dairy_skim"] + nutrition_df["dairy_low_fat"]

# VitB
nutrition_df["totalVitaminB_mg"] = nutrition_df["vitamin_b1_mg"] + nutrition_df["vitamin_b2_mg"] + nutrition_df["niacin_mg"] + nutrition_df["vitamin_b6_mg"] + nutrition_df["vitamin_b12_ug"] + nutrition_df["folicacid_ug"] 

In [ ]:
# CHUẨN BỊ NUTRI_SUBSET

nutri_subset = nutrition_df[[
    "sub_id",
    "energy_kcal",
    # nhóm protein 
    "crude_protein_g", 
    "EAA", 
    "NEAA", 
    "PLMF", 
    "PHSF", 
    # nhóm chất béo 
    "crude_fat_g", 
    "trans_fat", 
    "total_saturated_fatty_acids_g", 
    "total_monounsaturated_fatty_acids_g", 
    "total_polyunsaturated_fatty_acids_g", 
    "cholesterol_mg", 
    "oils_nuts_and_seeds", 
    # nhóm CH
    "total_carbohydrate_g", 
    "whole_grains_roots", 
    "monosaccharides", 
    "disaccharides", 
    # nhóm nước, trái cây, rau xanh
    "water_g", 
    "fruits", 
    "vegetables", 
    "dietary_fiber_g",
    # nhóm sữa
    "dairy_whole_fat", 
    "dairy_skim_low_fat", 
    # nhóm ions 
    "sodium_mg",
    "potassium_mg",
    "calcium_mg", 
    "magnesium_mg", 
    "phosphorus_mg",
    "iron_mg",
    "zinc_mg",
    # nhóm vitamin 
    "totalVitaminA_ui", 
    "totalVitaminB_mg",
    "vitaminc_mg", 
    "totalVitaminE_mg"
]].copy()

In [ ]:
# ĐỔI TÊN CỘT 

nutri_subset_names_mapper = {
    "energy_kcal": "Calories",
    # nhóm protein 
    "crude_protein_g": "Crude protein", 
    "EAA": "EAA", 
    "NEAA": "NEAA", 
    "PLMF": "PLMF", 
    "PHSF": "PHSF", 
    # nhóm chất béo 
    "crude_fat_g": "Crude fat", 
    "trans_fat": "Trans fat", 
    "total_saturated_fatty_acids_g": "TFA_S", 
    "total_monounsaturated_fatty_acids_g": "TFA_M", 
    "total_polyunsaturated_fatty_acids_g": "TFA_P", 
    "cholesterol_mg": "Cholesterol", 
    "oils_nuts_and_seeds": "Seeds", 
    # nhóm CH
    "total_carbohydrate_g": "Total carbohydrate", 
    "whole_grains_roots": "Whole Rhizome", 
    "monosaccharides": "Monosaccharides", 
    "disaccharides": "Disaccharides", 
    # nhóm nước, trái cây, rau xanh
    "water_g": "Water", 
    "fruits": "Fruit", 
    "vegetables": "Vegetable", 
    "dietary_fiber_g": "Dietary fiber",
    # nhóm sữa
    "dairy_whole_fat": "Dairy fullfat", 
    "dairy_skim_low_fat": "Dairy skim & lowfat", 
    # nhóm ions 
    "sodium_mg": "Sodium",
    "potassium_mg": "Potassium",
    "calcium_mg": "Calcium", 
    "magnesium_mg": "Magnesium", 
    "phosphorus_mg": "Phosphorus",
    "iron_mg": "Iron",
    "zinc_mg": "Zinc",
    # nhóm vitamin 
    "totalVitaminA_ui": "VitA", 
    "totalVitaminB_mg": "VitB", 
    "vitaminc_mg": "VitC",
    "totalVitaminE_mg": "VitE"
}

nutri_subset.rename(columns=nutri_subset_names_mapper, inplace=True)



In [ ]:
# TẠO FILE DESCRIPTION

from io import StringIO

cols = [c for c in nutri_subset.columns if c != "sub_id"]
descf = nutri_subset[cols].columns.to_frame().reset_index(drop=True)

# Dữ liệu gốc (bạn có thể thay bằng đường dẫn file CSV nếu có)
data = """family\tdescription
calories\tTotal energy intake 
protein\tTotal protein intake 
protein\tEssential amino acids 
protein\tNon-essential amino acids
protein\tProtein sources with low to moderate fat content
protein\tProtein sources with high to super-high fat content
fat\tTotal fat intake
fat\tTrans fatty acids
fat\tTotal saturated fatty acids.
fat\tTotal monounsaturated fatty acids
fat\tTotal polyunsaturated fatty acids
fat\tCholesterol intake
fat\tFat from seeds
carbohydrates\tTotal carbohydrate intake
carbohydrates\tWhole rhizome carbohydrates
carbohydrates\tSimple sugars
carbohydrates\tDouble sugars
water & fruits & vegetables\tWater intake
water & fruits & vegetables\tFruit intake
water & fruits & vegetables\tVegetable intake
water & fruits & vegetables\tDietary fiber intake
diary\tFull-fat dairy products
diary\tSkim-Lowfat dairy products
ions\tSodium intake
ions\tPotassium intake
ions\tCalcium intake
ions\tMagnesium intake
ions\tPhosphorus intake
ions\tIron intake
ions\tZinc intake
vitamins\tVitamin A
vitamins\tVitamin B
vitamins\tVitamin C
vitamins\tVitamin E
"""

# Đọc vào DataFrame
temp = pd.read_csv(StringIO(data), sep="\t")

# Viết hoa chữ cái đầu tiên của cột 'family'
descf["family"] = temp["family"].str.title()
descf["description"] = temp["description"]
descf.columns = ["exposure", "family", "description"]

descf.to_csv("../data/description.csv", sep=",", index=False, encoding="utf-8")
# descf.shape # (34, 3)

descf.head(5)


,exposure,family,description
0,Calories,Calories,Total energy intake
1,Crude protein,Protein,Total protein intake
2,EAA,Protein,Essential amino acids
3,NEAA,Protein,Non-essential amino acids
4,PLMF,Protein,Protein sources with low to moderate fat content


In [ ]:
# MERGE DATA

merged_df = pd.merge(imputed_subset, nutri_subset, how="inner", on="SubjectID").reset_index(drop=True)

exposures = merged_df[nutri_subset.columns] # exposures.shape # (237, 35)
# exposures.to_csv("../data/exposures.csv", sep=",", index=False, encoding="utf-8")

phenotype = merged_df[imputed_subset.columns] # phenotype.shape # (237, 6)
# phenotype.to_csv("../data/phenotype.csv", sep=",", index=False, encoding="utf-8")

display(exposures.head(5))
display(phenotype.head(5))

,number,Calories,Crude protein,EAA,NEAA,PLMF,PHSF,Crude fat,Trans fat,TFA_S,TFA_M,TFA_P,Cholesterol,Seeds,Total carbohydrate,Whole Rhizome,Monosaccharides,Disaccharides,Water,Fruit,Vegetable,Dietary fiber,Dairy (fullfat),Dairy (skim & lowfat),Sodium,Potassium,Calcium,Magnesium,Phosphorus,Iron,Zinc,VitA,VitB,VitC,VitE
0,14,2083.0610,124.6182,22201.3011,32826.6308,5.4371,1.4000,162.9716,525.2832,24569.6231,19792.7599,17017.9180,335.9144,6.3764,250.7860,8.6986,13.1918,24.8598,900.4350,1.7160,1.0443,6.5211,2.5243,0.0000,2390.1150,1891.7531,700.3972,176.5347,975.0997,5.3864,8.0484,2370.4715,159.0465,59.0759,20.4464
1,21,1426.0573,67.4991,19663.3002,29978.8721,5.5508,0.3500,46.8995,173.7764,9920.0503,8053.1767,5766.3122,325.3826,3.5027,172.7936,7.8940,5.4757,11.1657,569.9978,0.7357,2.2919,9.2719,0.4767,0.2500,2602.0115,2654.6762,991.9610,347.7083,853.9427,22.1991,10.0725,5820.9671,230.6884,45.7423,7.5912
2,22,1405.9154,57.7532,17229.8749,26613.9609,3.4917,0.6667,45.3218,368.3041,13538.3032,14460.8028,13996.0594,276.9586,3.4700,166.8673,7.5284,16.4401,19.5039,662.4117,1.5324,2.0227,10.0030,1.1870,0.0000,2477.4159,1761.5072,460.2836,173.8739,747.3765,7.2280,5.5922,7046.5492,157.8221,64.9104,24.6396
3,25,1361.9400,60.0294,19792.3091,35660.7755,1.8067,0.3333,44.3770,635.6100,17772.5463,12110.2063,8634.5342,79.3575,2.5433,182.6986,10.4153,4.7627,3.0464,336.5667,0.4154,0.9333,9.1559,2.3667,0.0000,1929.3325,964.6723,1251.6358,141.3843,799.6480,4.8395,8.0633,8654.7210,186.2692,61.0255,9.0621
4,26,2409.3350,113.4917,4736.5528,8180.0759,10.2344,0.0000,83.6643,210.5377,7744.0930,4150.8813,3154.1731,100.7036,9.2778,267.6278,16.9279,0.1988,7.1404,61.3762,0.0000,0.9000,0.8535,0.7028,0.0000,3686.9096,125.8711,215.1532,20.5541,188.5907,0.7045,1.8950,426.1897,23.4413,0.8203,4.0328


,number,sex,age,zbmi,calories(kcal),insulin_re
0,14,1,11.8200,0.9800,2083.0610,no
1,21,0,12.7200,0.5400,1426.0573,no
2,22,0,9.6800,0.3700,1405.9154,yes
3,25,0,12.3700,-0.3800,1361.9400,no
4,26,1,9.7300,-0.8800,2409.3350,no
